In [ ]:
from model.calibration import fit_model, check_fit
from model.optimisation import optimise_interventions
from plotting.plots import plot_future_trajectories, make_intervention_piechart

minimised_outcomes = {
    "incidence_per100k": "TB incidence in 2040",
    # "ltbi_prevalence_perc": "ltbi",
    # "tb_deaths": "TB mortality in 2040",
    "cumulative_future_deaths": "TB deaths (2024-2040)",
    # "cumulative_future_paed_deaths": "Paediatric TB deaths (2024-2040)",
    # "years_of_life_lost": "Years of life lost (2025-2040)",
}


def run_analysis(target_incidence=100):

    bcm, mle_params = fit_model(target_incidence=target_incidence)
    check_fit(bcm, mle_params)
    derived_outputs = {"baseline": bcm.run(mle_params | {"decision_var_trans": 0., "decision_var_cdr": 0., "decision_var_pt": 0.}).derived_outputs}

    opti_decision_vars = {}
    for minimised_indicator in minimised_outcomes:
        opti_bcm, opti_params = optimise_interventions(mle_params, minimised_indicator=minimised_indicator)
        res = opti_bcm.run(opti_params)
        derived_outputs[minimised_indicator] = res.derived_outputs
        opti_decision_vars[minimised_indicator] = opti_params

    return derived_outputs, opti_decision_vars

In [ ]:
master_derived_outputs, master_opti_decision_vars = {}, {}
for incidence in [50, 100, 200, 500, 1000]:
    derived_outputs, opti_decision_vars = run_analysis(target_incidence=incidence)
    master_derived_outputs[incidence] = derived_outputs
    master_opti_decision_vars[incidence] = opti_decision_vars


In [ ]:
master_opti_decision_vars[200]

In [ ]:
def get_main_numbers(master_derived_outputs, low_inc=200, high_inc=1000):
    for incidence in [low_inc, high_inc]:
        print(f"incidence: {incidence}")
        derived_outputs = master_derived_outputs[incidence]

        mortality_when_min_mortality = derived_outputs["cumulative_future_deaths"]["cumulative_future_deaths"].loc[2040]
        mortality_when_min_incidence = derived_outputs["incidence_per100k"]["cumulative_future_deaths"].loc[2040]
        perc_greater = 100. * (mortality_when_min_incidence - mortality_when_min_mortality) / mortality_when_min_mortality
        print(f"optimising for TB incidence led to an estimated {round(perc_greater)}% higher cumulative TB mortality compared to minimising cumulative mortality") 

        incidence_when_min_mortality = derived_outputs["cumulative_future_deaths"]["incidence_per100k"].loc[2040]
        incidence_when_min_incidence = derived_outputs["incidence_per100k"]["incidence_per100k"].loc[2040]
        perc_greater = 100. * (incidence_when_min_mortality - incidence_when_min_incidence) / incidence_when_min_incidence
        print(f"TB incidence in 2040 was only {round(perc_greater)}% higher when minimising cumulative mortality") 


In [ ]:
get_main_numbers(master_derived_outputs, 100, 500)

In [ ]:
get_main_numbers(master_derived_outputs, 100, 500)

In [ ]:
master_derived_outputs[100].keys()

In [ ]:
from matplotlib import pyplot as plt

sc_titles = {'baseline': 'no intervention', 'incidence_per100k': 'minimising incidence', 'cumulative_future_deaths': 'minimising cumulative deaths'}
 
sc_titles_split = {'incidence_per100k': 'minimising\nincidence', 'cumulative_future_deaths': 'minimising\ncumulative\ndeaths'}
sc_colors = {'baseline': 'black', 'incidence_per100k': 'tomato', 'cumulative_future_deaths': 'cornflowerblue'}
ls = {'baseline': '--', 'incidence_per100k': '-', 'cumulative_future_deaths': '-'}

def plot_abstract_figure(derived_outputs, output="incidence_per100k"):
    fig, axs = plt.subplots(1, 2, figsize=(6, 3.5), gridspec_kw={'width_ratios': [5, 2]})

    # Add content to the subplots (replace with your data)
    xmin = 2023
    ymax = 0.
    for sc_name, derived_df in derived_outputs.items():
        derived_df[output].loc[xmin:].plot(label=sc_titles[sc_name], ax=axs[0], color=sc_colors[sc_name], linestyle=ls[sc_name])
        ymax = max(ymax, derived_df[output].loc[xmin:].max())

    axs[0].set_ylabel(output_names[output])
    axs[0].set_ylim((0, 1.55 * ymax))
    axs[0].legend()

    # Bar plot
    names = [sc_titles_split[sc_name] for sc_name in derived_outputs if sc_name != 'baseline']
    values = [derived_outputs[sc_name]['cumulative_future_deaths'].loc[2040] for sc_name in derived_outputs if sc_name != 'baseline']
    colors = [sc_colors[sc_name] for sc_name in derived_outputs if sc_name != 'baseline']

    axs[1].bar(names, values, color=colors)
    axs[1].set_ylabel('TB deaths over 2025-2040')
    plt.xticks(rotation=45)
    # axs[1].set_title('Panel 2')

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Show the plot
    # plt.show()
    plt.savefig('abstract_figure.png', dpi=100) 

plot_abstract_figure(master_derived_outputs[100])


In [ ]:
derived_outputs, opti_decision_vars = run_analysis(target_incidence=50)

In [ ]:
opti_decision_vars

In [ ]:
for sc, opti_vars in opti_decision_vars.items():
    print(sc)
    print(sum(opti_vars.values()))

In [ ]:
output_names = {
    "incidence_per100k": "TB incidence (/100,000/y)",
    "ltbi_prevalence_perc": "LTBI prevalence (%)",
    "cumulative_future_deaths": "Cumulative TB deaths",
    "cumulative_future_paed_deaths": "Cumulative paediatric TB deaths",

}

from matplotlib import pyplot as plt


def plot_optimised_trajectories(derived_outputs, output="incidence_per100k", ax=None):

    if not ax:
        fig, ax = plt.subplots(1, 1)

    xmin = 2020
    ymax = 0.
    for sc_name, derived_df in derived_outputs.items():
        derived_df[output].loc[xmin:].plot(label=sc_name, )
        ymax = max(ymax, derived_df[output].loc[xmin:].max())

    ax.set_ylabel(output_names[output])
    ax.set_ylim((0, 1.2 * ymax))
    ax.legend()


plot_optimised_trajectories(derived_outputs)

In [ ]:
plot_optimised_trajectories(master_derived_outputs[100], output="incidence_per100k")
plot_optimised_trajectories(master_derived_outputs[100], output="cumulative_future_deaths")

In [ ]:
plot_optimised_trajectories(master_derived_outputs[500], output="incidence_per100k")
plot_optimised_trajectories(master_derived_outputs[500], output="cumulative_future_deaths")

In [ ]:
from plotting.plots import make_intervention_piechart
for sc_name, opti_vars in opti_decision_vars.items():
    ax = make_intervention_piechart(opti_vars)
    ax.set_title(sc_name)

In [ ]:
# relative difference in incidence
def get_relative_diff(derived_outputs, output, scenarios=["incidence_per100k", "cumulative_future_deaths"]):

    output_1 = derived_outputs[scenarios[1]][output].loc[2040]
    output_0 = derived_outputs[scenarios[0]][output].loc[2040]

    print(output)
    for sc in scenarios:
        print(f"Minimising {sc}: {derived_outputs[sc][output].loc[2040]}")


    return 100 * (output_1 - output_0) / output_0



print(get_relative_diff(derived_outputs, "incidence_per100k", ["cumulative_future_deaths", "incidence_per100k"]))
print()
print(get_relative_diff(derived_outputs, "cumulative_future_deaths", ["incidence_per100k", "cumulative_future_deaths"]))

